In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent
elif PROJECT_ROOT.name.startswith("0"):
    PROJECT_ROOT = PROJECT_ROOT.parent.parent

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

%load_ext autoreload
%autoreload 2

In [3]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn import model_selection
import xgboost as xgb

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from match_forecast.utils import *

In [4]:
PROJ_ROOT = Path("QRT-Challenge-2024").resolve().parents[1]
DATA_DIR = PROJ_ROOT / "data"
RAW_DATA_DIR = DATA_DIR / "raw"
PROCESSED_DATA_DIR = DATA_DIR / "processed"

In [5]:
try:
    test_home_team_statistics_df = pd.read_csv(RAW_DATA_DIR / "test_home_team_statistics_df.csv", index_col=0)
    test_away_team_statistics_df = pd.read_csv(RAW_DATA_DIR / "test_away_team_statistics_df.csv", index_col=0)
    print("Files loaded")
    
except FileNotFoundError as e:
    print(e)

Files loaded


In [6]:
home_teams = test_home_team_statistics_df
away_teams = test_away_team_statistics_df

# Rename
home_teams.columns = 'HOME_' + home_teams.columns
away_teams.columns = 'AWAY_' + away_teams.columns

test_data_teams = pd.concat([home_teams, away_teams], join='inner', axis=1)
test_data_teams = test_data_teams.apply(replace_null_values)

print(test_data_teams.shape) 

(25368, 280)


In [7]:
test_data_teams = test_data_teams.fillna(test_data_teams.select_dtypes(include=['number']).median())

In [8]:
### DIFFERENCES
# Get the list of HOME and AWAY columns
home_columns = [col for col in test_data_teams.columns if col.startswith('HOME_TEAM_')]
away_columns = [col for col in test_data_teams.columns if col.startswith('AWAY_TEAM_')]

# Ensure there is a corresponding AWAY column for each HOME column
common_columns = [(home, home.replace('HOME_TEAM_', 'AWAY_TEAM_')) for home in home_columns if home.replace('HOME_TEAM_', 'AWAY_TEAM_') in away_columns]

# Add difference features
for home_col, away_col in common_columns:
    diff_col_name = home_col.replace('HOME_TEAM_', 'DIFF_')  # Name of the new column
    test_data_teams[diff_col_name] = test_data_teams[home_col] - test_data_teams[away_col]

# Delete initial features
cols_to_drop = home_columns + away_columns
test_data_teams.drop(columns=cols_to_drop, inplace=True)


In [9]:
test_data_teams.shape

(25368, 140)

In [10]:
test_data_teams.head()

,DIFF_SHOTS_TOTAL_season_sum,DIFF_SHOTS_INSIDEBOX_season_sum,DIFF_SHOTS_OFF_TARGET_season_sum,DIFF_SHOTS_ON_TARGET_season_sum,DIFF_SHOTS_OUTSIDEBOX_season_sum,DIFF_PASSES_season_sum,DIFF_SUCCESSFUL_PASSES_season_sum,DIFF_SAVES_season_sum,DIFF_CORNERS_season_sum,DIFF_FOULS_season_sum,DIFF_YELLOWCARDS_season_sum,DIFF_REDCARDS_season_sum,DIFF_OFFSIDES_season_sum,DIFF_ATTACKS_season_sum,DIFF_PENALTIES_season_sum,DIFF_SUBSTITUTIONS_season_sum,DIFF_BALL_SAFE_season_sum,DIFF_DANGEROUS_ATTACKS_season_sum,DIFF_INJURIES_season_sum,DIFF_GOALS_season_sum,DIFF_GAME_WON_season_sum,DIFF_GAME_DRAW_season_sum,DIFF_GAME_LOST_season_sum,DIFF_SHOTS_TOTAL_season_average,DIFF_SHOTS_INSIDEBOX_season_average,DIFF_SHOTS_OFF_TARGET_season_average,DIFF_SHOTS_ON_TARGET_season_average,DIFF_SHOTS_OUTSIDEBOX_season_average,DIFF_PASSES_season_average,DIFF_SUCCESSFUL_PASSES_season_average,DIFF_SUCCESSFUL_PASSES_PERCENTAGE_season_average,DIFF_SAVES_season_average,DIFF_CORNERS_season_average,DIFF_BALL_POSSESSION_season_average,DIFF_FOULS_season_average,DIFF_YELLOWCARDS_season_average,DIFF_REDCARDS_season_average,DIFF_OFFSIDES_season_average,DIFF_ATTACKS_season_average,DIFF_PENALTIES_season_average,DIFF_SUBSTITUTIONS_season_average,DIFF_BALL_SAFE_season_average,DIFF_DANGEROUS_ATTACKS_season_average,DIFF_INJURIES_season_average,DIFF_GOALS_season_average,DIFF_GAME_WON_season_average,DIFF_GAME_DRAW_season_average,DIFF_GAME_LOST_season_average,DIFF_SHOTS_TOTAL_season_std,DIFF_SHOTS_INSIDEBOX_season_std,DIFF_SHOTS_OFF_TARGET_season_std,DIFF_SHOTS_ON_TARGET_season_std,DIFF_SHOTS_OUTSIDEBOX_season_std,DIFF_PASSES_season_std,DIFF_SUCCESSFUL_PASSES_season_std,DIFF_SUCCESSFUL_PASSES_PERCENTAGE_season_std,DIFF_SAVES_season_std,DIFF_CORNERS_season_std,DIFF_BALL_POSSESSION_season_std,DIFF_FOULS_season_std,DIFF_YELLOWCARDS_season_std,DIFF_REDCARDS_season_std,DIFF_OFFSIDES_season_std,DIFF_ATTACKS_season_std,DIFF_PENALTIES_season_std,DIFF_SUBSTITUTIONS_season_std,DIFF_BALL_SAFE_season_std,DIFF_DANGEROUS_ATTACKS_season_std,DIFF_INJURIES_season_std,DIFF_GOALS_season_std,DIFF_SHOTS_TOTAL_5_last_match_sum,DIFF_SHOTS_INSIDEBOX_5_last_match_sum,DIFF_SHOTS_OFF_TARGET_5_last_match_sum,DIFF_SHOTS_ON_TARGET_5_last_match_sum,DIFF_SHOTS_OUTSIDEBOX_5_last_match_sum,DIFF_PASSES_5_last_match_sum,DIFF_SUCCESSFUL_PASSES_5_last_match_sum,DIFF_SAVES_5_last_match_sum,DIFF_CORNERS_5_last_match_sum,DIFF_FOULS_5_last_match_sum,DIFF_YELLOWCARDS_5_last_match_sum,DIFF_REDCARDS_5_last_match_sum,DIFF_OFFSIDES_5_last_match_sum,DIFF_ATTACKS_5_last_match_sum,DIFF_PENALTIES_5_last_match_sum,DIFF_SUBSTITUTIONS_5_last_match_sum,DIFF_BALL_SAFE_5_last_match_sum,DIFF_DANGEROUS_ATTACKS_5_last_match_sum,DIFF_INJURIES_5_last_match_sum,DIFF_GOALS_5_last_match_sum,DIFF_GAME_WON_5_last_match_sum,DIFF_GAME_DRAW_5_last_match_sum,DIFF_GAME_LOST_5_last_match_sum,DIFF_SHOTS_TOTAL_5_last_match_average,DIFF_SHOTS_INSIDEBOX_5_last_match_average,DIFF_SHOTS_OFF_TARGET_5_last_match_average,DIFF_SHOTS_ON_TARGET_5_last_match_average,DIFF_SHOTS_OUTSIDEBOX_5_last_match_average,DIFF_PASSES_5_last_match_average,DIFF_SUCCESSFUL_PASSES_5_last_match_average,DIFF_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_average,DIFF_SAVES_5_last_match_average,DIFF_CORNERS_5_last_match_average,DIFF_BALL_POSSESSION_5_last_match_average,DIFF_FOULS_5_last_match_average,DIFF_YELLOWCARDS_5_last_match_average,DIFF_REDCARDS_5_last_match_average,DIFF_OFFSIDES_5_last_match_average,DIFF_ATTACKS_5_last_match_average,DIFF_PENALTIES_5_last_match_average,DIFF_SUBSTITUTIONS_5_last_match_average,DIFF_BALL_SAFE_5_last_match_average,DIFF_DANGEROUS_ATTACKS_5_last_match_average,DIFF_INJURIES_5_last_match_average,DIFF_GOALS_5_last_match_average,DIFF_GAME_WON_5_last_match_average,DIFF_GAME_DRAW_5_last_match_average,DIFF_GAME_LOST_5_last_match_average,DIFF_SHOTS_TOTAL_5_last_match_std,DIFF_SHOTS_INSIDEBOX_5_last_match_std,DIFF_SHOTS_OFF_TARGET_5_last_match_std,DIFF_SHOTS_ON_TARGET_5_last_match_std,DIFF_SHOTS_OUTSIDEBOX_5_last_match_std,DIFF_PASSES_5_last_match_std,DIFF_SUCCESSFUL

In [12]:
test_data_teams_avg = test_data_teams.loc[:, test_data_teams.columns.str.endswith('_average')]
print(test_data_teams_avg.shape)

test_data_teams_sum = test_data_teams.loc[:, test_data_teams.columns.str.endswith('_sum')]
print(test_data_teams_sum.shape)

test_data_teams_std = test_data_teams.loc[:, test_data_teams.columns.str.endswith('_std')]
print(test_data_teams_std.shape)

(25368, 50)
(25368, 46)
(25368, 44)


In [13]:
try:
    test_home_player_statistics_df = pd.read_csv(RAW_DATA_DIR / "test_home_player_statistics_df.csv", index_col=0)
    test_away_player_statistics_df = pd.read_csv(RAW_DATA_DIR / "test_away_player_statistics_df.csv", index_col=0)
    print("Files loaded")
    
except FileNotFoundError as e:
    print(e)

Files loaded


In [14]:
home_players = test_home_player_statistics_df
away_players = test_away_player_statistics_df

# Rename
home_players.columns = 'HOME_' + home_players.columns
away_players.columns = 'AWAY_' + away_players.columns

home_players = home_players.apply(replace_null_values)
away_players = away_players.apply(replace_null_values)

print(home_players.shape)
print(away_players.shape) 

(509816, 303)
(504626, 303)


In [15]:
home_players = home_players.fillna({'HOME_POSITION': home_players['HOME_POSITION'].mode()[0]})
away_players = away_players.fillna({'AWAY_POSITION': away_players['AWAY_POSITION'].mode()[0]})

In [16]:
position_map = {
    'attacker': 'offensive',
    'midfielder': 'offensive',
    'defender': 'defender',
    'goalkeeper': 'goalkeeper'
}

## Average

In [17]:
columns_to_aggregate_home = [col for col in home_players.columns if col.endswith(('average'))]
columns_to_aggregate_away = [col for col in away_players.columns if col.endswith(('average'))]

# Building tables
home_agg = home_players.groupby([home_players.index, 'HOME_POSITION']).agg({
    col: ['mean'] for col in columns_to_aggregate_home  # mean agg
}).reset_index()

away_agg = away_players.groupby([away_players.index, 'AWAY_POSITION']).agg({
    col: ['mean'] for col in columns_to_aggregate_away  # mean agg
}).reset_index()

# Flatten multi-level columns from previous groupby
home_agg.columns = ['_'.join(col).strip() for col in home_agg.columns.values]
home_agg = home_agg.rename(columns={'ID_' : 'HOME_ID', 'HOME_POSITION_' : 'HOME_POSITION'})

away_agg.columns = ['_'.join(col).strip() for col in away_agg.columns.values]
away_agg = away_agg.rename(columns={'ID_' : 'AWAY_ID', 'AWAY_POSITION_' : 'AWAY_POSITION'})

# For home aggregation:
home_agg['position_group'] = home_agg['HOME_POSITION'].map(position_map)  # Map positions to groups
agg_columns = [col for col in home_agg.columns 
               if home_agg[col].dtype in ['float64', 'int64'] and col != 'HOME_ID']  # Select numeric columns
home_group = home_agg.groupby(['HOME_ID', 'position_group'])[agg_columns].mean().reset_index()  # Compute mean for each group

home_pivot = home_group.pivot(index='HOME_ID', columns='position_group')  # Pivot to get separate columns per group
home_pivot.columns = ['_'.join(col).strip() for col in home_pivot.columns.values]  # Flatten multi-level column index
home_pivot.reset_index(inplace=True)

# For away aggregation:
away_agg['position_group'] = away_agg['AWAY_POSITION'].map(position_map)  # Map positions to groups
agg_columns = [col for col in away_agg.columns 
               if away_agg[col].dtype in ['float64', 'int64'] and col != 'AWAY_ID']  # Select numeric columns
away_group = away_agg.groupby(['AWAY_ID', 'position_group'])[agg_columns].mean().reset_index()  # Compute mean for each group

away_pivot = away_group.pivot(index='AWAY_ID', columns='position_group')  # Pivot to get separate columns per group
away_pivot.columns = ['_'.join(col).strip() for col in away_pivot.columns.values]  # Flatten multi-level column index
away_pivot.reset_index(inplace=True)

# Merge
home_pivot.rename(columns={'HOME_ID': 'ID'}, inplace=True)
away_pivot.rename(columns={'AWAY_ID': 'ID'}, inplace=True)
test_data_players_avg = home_pivot.merge(away_pivot, on='ID', how='inner')
test_data_players_avg.set_index('ID', inplace=True)

test_data_players_avg = test_data_players_avg.drop(columns=test_data_players_avg.columns[test_data_players_avg.isna().mean() > 0.5])
test_data_players_avg = test_data_players_avg.fillna(test_data_players_avg.select_dtypes(include=['number']).median())

### DIFFERENCES
home_columns = [col for col in test_data_players_avg.columns if col.startswith('HOME_PLAYER_')]
away_columns = [col for col in test_data_players_avg.columns if col.startswith('AWAY_PLAYER_')]

common_columns = [(home, home.replace('HOME_PLAYER_', 'AWAY_PLAYER_')) for home in home_columns if home.replace('HOME_PLAYER_', 'AWAY_PLAYER_') in away_columns]

for home_col, away_col in common_columns:
    diff_col_name = home_col.replace('HOME_PLAYER_', 'DIFF_PLAYER_') 
    test_data_players_avg[diff_col_name] = test_data_players_avg[home_col] - test_data_players_avg[away_col]

cols_to_drop = home_columns + away_columns
test_data_players_avg.drop(columns=cols_to_drop, inplace=True)


## Sum

In [18]:
# columns_to_aggregate_home = [col for col in home_players.columns if col.endswith(('sum'))]
# columns_to_aggregate_away = [col for col in away_players.columns if col.endswith(('sum'))]

# # Building tables
# home_agg = home_players.groupby([home_players.index, 'HOME_POSITION']).agg({
#     col: ['mean'] for col in columns_to_aggregate_home  # mean agg
# }).reset_index()

# away_agg = away_players.groupby([away_players.index, 'AWAY_POSITION']).agg({
#     col: ['mean'] for col in columns_to_aggregate_away  # mean agg
# }).reset_index()

# # Flatten multi-level columns from previous groupby
# home_agg.columns = ['_'.join(col).strip() for col in home_agg.columns.values]
# home_agg = home_agg.rename(columns={'ID_' : 'HOME_ID', 'HOME_POSITION_' : 'HOME_POSITION'})

# away_agg.columns = ['_'.join(col).strip() for col in away_agg.columns.values]
# away_agg = away_agg.rename(columns={'ID_' : 'AWAY_ID', 'AWAY_POSITION_' : 'AWAY_POSITION'})

# # For home aggregation:
# home_agg['position_group'] = home_agg['HOME_POSITION'].map(position_map)  # Map positions to groups
# agg_columns = [col for col in home_agg.columns 
#                if home_agg[col].dtype in ['float64', 'int64'] and col != 'HOME_ID']  # Select numeric columns
# home_group = home_agg.groupby(['HOME_ID', 'position_group'])[agg_columns].mean().reset_index()  # Compute mean for each group

# home_pivot = home_group.pivot(index='HOME_ID', columns='position_group')  # Pivot to get separate columns per group
# home_pivot.columns = ['_'.join(col).strip() for col in home_pivot.columns.values]  # Flatten multi-level column index
# home_pivot.reset_index(inplace=True)

# # For away aggregation:
# away_agg['position_group'] = away_agg['AWAY_POSITION'].map(position_map)  # Map positions to groups
# agg_columns = [col for col in away_agg.columns 
#                if away_agg[col].dtype in ['float64', 'int64'] and col != 'AWAY_ID']  # Select numeric columns
# away_group = away_agg.groupby(['AWAY_ID', 'position_group'])[agg_columns].mean().reset_index()  # Compute mean for each group

# away_pivot = away_group.pivot(index='AWAY_ID', columns='position_group')  # Pivot to get separate columns per group
# away_pivot.columns = ['_'.join(col).strip() for col in away_pivot.columns.values]  # Flatten multi-level column index
# away_pivot.reset_index(inplace=True)

# # Merge
# home_pivot.rename(columns={'HOME_ID': 'ID'}, inplace=True)
# away_pivot.rename(columns={'AWAY_ID': 'ID'}, inplace=True)
# test_data_players_sum = home_pivot.merge(away_pivot, on='ID', how='inner')
# test_data_players_sum.set_index('ID', inplace=True)

# test_data_players_sum = test_data_players_sum.drop(columns=test_data_players_sum.columns[test_data_players_sum.isna().mean() > 0.5])
# test_data_players_sum = test_data_players_sum.fillna(test_data_players_sum.select_dtypes(include=['number']).median())

# ### DIFFERENCES
# home_columns = [col for col in test_data_players_sum.columns if col.startswith('HOME_PLAYER_')]
# away_columns = [col for col in test_data_players_sum.columns if col.startswith('AWAY_PLAYER_')]

# common_columns = [(home, home.replace('HOME_PLAYER_', 'AWAY_PLAYER_')) for home in home_columns if home.replace('HOME_PLAYER_', 'AWAY_PLAYER_') in away_columns]

# for home_col, away_col in common_columns:
#     diff_col_name = home_col.replace('HOME_PLAYER_', 'DIFF_PLAYER_') 
#     test_data_players_sum[diff_col_name] = test_data_players_sum[home_col] - test_data_players_sum[away_col]

# cols_to_drop = home_columns + away_columns
# test_data_players_sum.drop(columns=cols_to_drop, inplace=True)

In [19]:
# test_data_players_interim = test_data_players_sum.merge(test_data_players_avg, left_index=True, right_index=True, how='inner')

## Std

In [20]:
# columns_to_aggregate_home = [col for col in home_players.columns if col.endswith(('std'))]
# columns_to_aggregate_away = [col for col in away_players.columns if col.endswith(('std'))]

# # Building tables
# home_agg = home_players.groupby([home_players.index, 'HOME_POSITION']).agg({
#     col: ['mean'] for col in columns_to_aggregate_home  # mean agg
# }).reset_index()

# away_agg = away_players.groupby([away_players.index, 'AWAY_POSITION']).agg({
#     col: ['mean'] for col in columns_to_aggregate_away  # mean agg
# }).reset_index()

# # Flatten multi-level columns from previous groupby
# home_agg.columns = ['_'.join(col).strip() for col in home_agg.columns.values]
# home_agg = home_agg.rename(columns={'ID_' : 'HOME_ID', 'HOME_POSITION_' : 'HOME_POSITION'})

# away_agg.columns = ['_'.join(col).strip() for col in away_agg.columns.values]
# away_agg = away_agg.rename(columns={'ID_' : 'AWAY_ID', 'AWAY_POSITION_' : 'AWAY_POSITION'})

# # For home aggregation:
# home_agg['position_group'] = home_agg['HOME_POSITION'].map(position_map)  # Map positions to groups
# agg_columns = [col for col in home_agg.columns 
#                if home_agg[col].dtype in ['float64', 'int64'] and col != 'HOME_ID']  # Select numeric columns
# home_group = home_agg.groupby(['HOME_ID', 'position_group'])[agg_columns].mean().reset_index()  # Compute mean for each group

# home_pivot = home_group.pivot(index='HOME_ID', columns='position_group')  # Pivot to get separate columns per group
# home_pivot.columns = ['_'.join(col).strip() for col in home_pivot.columns.values]  # Flatten multi-level column index
# home_pivot.reset_index(inplace=True)

# # For away aggregation:
# away_agg['position_group'] = away_agg['AWAY_POSITION'].map(position_map)  # Map positions to groups
# agg_columns = [col for col in away_agg.columns 
#                if away_agg[col].dtype in ['float64', 'int64'] and col != 'AWAY_ID']  # Select numeric columns
# away_group = away_agg.groupby(['AWAY_ID', 'position_group'])[agg_columns].mean().reset_index()  # Compute mean for each group

# away_pivot = away_group.pivot(index='AWAY_ID', columns='position_group')  # Pivot to get separate columns per group
# away_pivot.columns = ['_'.join(col).strip() for col in away_pivot.columns.values]  # Flatten multi-level column index
# away_pivot.reset_index(inplace=True)

# # Merge
# home_pivot.rename(columns={'HOME_ID': 'ID'}, inplace=True)
# away_pivot.rename(columns={'AWAY_ID': 'ID'}, inplace=True)
# test_data_players_std = home_pivot.merge(away_pivot, on='ID', how='inner')
# test_data_players_std.set_index('ID', inplace=True)

# test_data_players_std = test_data_players_std.drop(columns=test_data_players_std.columns[test_data_players_std.isna().mean() > 0.5])
# test_data_players_std = test_data_players_std.fillna(test_data_players_std.select_dtypes(include=['number']).median())

# ### DIFFERENCES
# home_columns = [col for col in test_data_players_std.columns if col.startswith('HOME_PLAYER_')]
# away_columns = [col for col in test_data_players_std.columns if col.startswith('AWAY_PLAYER_')]

# common_columns = [(home, home.replace('HOME_PLAYER_', 'AWAY_PLAYER_')) for home in home_columns if home.replace('HOME_PLAYER_', 'AWAY_PLAYER_') in away_columns]

# for home_col, away_col in common_columns:
#     diff_col_name = home_col.replace('HOME_PLAYER_', 'DIFF_PLAYER_') 
#     test_data_players_std[diff_col_name] = test_data_players_std[home_col] - test_data_players_std[away_col]

# cols_to_drop = home_columns + away_columns
# test_data_players_std.drop(columns=cols_to_drop, inplace=True)

In [21]:
# test_data_players = test_data_players_interim.merge(test_data_players_std, left_index=True, right_index=True, how='inner')

In [22]:
test_data_players_avg.shape

(25368, 282)

## Final dataset

In [23]:
test_data = test_data_teams_avg.merge(test_data_players_avg, left_index=True, right_index=True, how='inner')

In [24]:
test_data.shape

(25368, 332)

In [25]:
try:
    train_data = pd.read_csv(PROCESSED_DATA_DIR / "train_data.csv", index_col=0)
    print("Files loaded")
    
except FileNotFoundError as e:
    print(e)

Files loaded


In [26]:
common = train_data.columns.intersection(test_data.columns)
test_aligned = test_data[common]
test_aligned.shape

(25368, 275)

In [27]:
print_perc_nans(test_aligned, threshold=50)

No columns have a percentage of NaN values above the threshold.


In [28]:
test_aligned = test_aligned.drop(columns=test_aligned.columns[test_aligned.isna().mean() > 0.5])

In [29]:
print_perc_nans(test_aligned, threshold=0)

No columns have a percentage of NaN values above the threshold.


In [30]:
test_aligned = test_aligned.fillna(test_aligned.select_dtypes(include=['number']).median())

In [33]:
test_aligned.head()

,DIFF_GAME_LOST_season_average,DIFF_PLAYER_GOALS_CONCEDED_season_average_mean_offensive,DIFF_SHOTS_TOTAL_season_average,DIFF_BALL_POSSESSION_season_average,DIFF_SHOTS_INSIDEBOX_season_average,DIFF_SHOTS_ON_TARGET_season_average,DIFF_GAME_WON_season_average,DIFF_PLAYER_GOALS_CONCEDED_season_average_mean_goalkeeper,DIFF_PLAYER_GOALS_CONCEDED_season_average_mean_defender,DIFF_DANGEROUS_ATTACKS_season_average,DIFF_PLAYER_BIG_CHANCES_CREATED_season_average_mean_offensive,DIFF_GOALS_season_average,DIFF_PLAYER_ASSISTS_season_average_mean_offensive,DIFF_PLAYER_KEY_PASSES_season_average_mean_offensive,DIFF_SHOTS_OFF_TARGET_season_average,DIFF_BALL_POSSESSION_5_last_match_average,DIFF_ATTACKS_season_average,DIFF_CORNERS_season_average,DIFF_PLAYER_ASSISTS_season_average_mean_defender,DIFF_PLAYER_GOALS_season_average_mean_offensive,DIFF_SAVES_season_average,DIFF_SUCCESSFUL_PASSES_season_average,DIFF_PLAYER_ACCURATE_PASSES_season_average_mean_defender,DIFF_PLAYER_GOALS_CONCEDED_5_last_match_average_mean_offensive,DIFF_PASSES_season_average,DIFF_PLAYER_KEY_PASSES_5_last_match_average_mean_offensive,DIFF_PLAYER_ACCURATE_PASSES_season_average_mean_offensive,DIFF_PLAYER_PASSES_season_average_mean_offensive,DIFF_PLAYER_GOALS_CONCEDED_5_last_match_average_mean_defender,DIFF_PLAYER_KEY_PASSES_season_average_mean_defender,DIFF_PLAYER_CLEARANCES_5_last_match_average_mean_offensive,DIFF_PLAYER_ACCURATE_PASSES_PERCENTAGE_season_average_mean_defender,DIFF_PLAYER_GOALS_CONCEDED_5_last_match_average_mean_goalkeeper,DIFF_PLAYER_SHOTS_ON_TARGET_season_average_mean_offensive,DIFF_PLAYER_SHOTS_BLOCKED_season_average_mean_offensive,DIFF_SHOTS_TOTAL_5_last_match_average,DIFF_PLAYER_PASSES_season_average_mean_defender,DIFF_PLAYER_SHOTS_TOTAL_season_average_mean_offensive,DIFF_PLAYER_ACCURATE_PASSES_PERCENTAGE_season_average_mean_offensive,DIFF_PLAYER_BIG_CHANCES_MISSED_season_average_mean_offensive,DIFF_PLAYER_CLEARANCES_season_average_mean_offensive,DIFF_PLAYER_BLOCKED_SHOTS_season_average_mean_defender,DIFF_CORNERS_5_last_match_average,DIFF_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_average,DIFF_PLAYER_CLEARANCES_season_average_mean_defender,DIFF_PASSES_5_last_match_average,DIFF_PLAYER_ACCURATE_PASSES_PERCENTAGE_5_last_match_average_mean_defender,DIFF_SHOTS_INSIDEBOX_5_last_match_average,DIFF_PLAYER_ACCURATE_PASSES_5_last_match_average_mean_offensive,DIFF_PLAYER_BIG_CHANCES_CREATED_5_last_match_average_mean_offensive,DIFF_PLAYER_ACCURATE_PASSES_PERCENTAGE_5_last_match_average_mean_goalkeeper,DIFF_SUCCESSFUL_PASSES_5_last_match_average,DIFF_PLAYER_REDCARDS_season_average_mean_offensive,DIFF_PLAYER_ACCURATE_PASSES_5_last_match_average_mean_defender,DIFF_GAME_WON_5_last_match_average,DIFF_SHOTS_ON_TARGET_5_last_match_average,DIFF_DANGEROUS_ATTACKS_5_last_match_average,DIFF_PLAYER_BLOCKED_SHOTS_5_last_match_average_mean_defender,DIFF_PLAYER_SAVES_INSIDE_BOX_5_last_match_average_mean_goalkeeper,DIFF_ATTACKS_5_last_match_average,DIFF_PLAYER_PASSES_5_last_match_average_mean_offensive,DIFF_PLAYER_GOALKEEPER_GOALS_CONCEDED_season_average_mean_goalkeeper,DIFF_PLAYER_CLEARANCES_5_last_match_average_mean_defender,DIFF_PLAYER_SHOTS_TOTAL_5_last_match_average_mean_offensive,DIFF_SUCCESSFUL_PASSES_PERCENTAGE_season_average,DIFF_PLAYER_ASSISTS_5_last_match_average_mean_defender,DIFF_PLAYER_SAVES_5_last_match_average_mean_goalkeeper,DIFF_GOALS_5_last_match_average,DIFF_SAVES_5_last_match_average,DIFF_PLAYER_YELLOWCARDS_5_last_match_average_mean_defender,DIFF_PLAYER_GOALS_5_last_match_average_mean_offensive,DIFF_PLAYER_SHOTS_ON_TARGET_5_last_match_average_mean_offensive,DIFF_PLAYER_ASSISTS_5_last_match_average_mean_offensive,DIFF_PLAYER_YELLOWCARDS_5_last_match_average_mean_offensive,DIFF_PLAYER_RATING_season_average_mean_offensive,DIFF_PLAYER_PENALTIES_WON_season_average_mean_offensive,DIFF_OFFSIDES_season_average,DIFF_SHOTS_OFF_TARGET_5_last_match_average,DIFF_PLAYER_PENALTIES_COMMITTED_season_average_mean_defender,DIFF_PLAYER_OFFSIDES_season_average_mean_offensive,DIFF_PLAY

In [34]:
# Store test data

test_data_path = PROCESSED_DATA_DIR / "test_data.csv"
test_aligned.to_csv(test_data_path, index=True)